<a href="https://colab.research.google.com/github/Augustus2001/Datascience/blob/main/Monkeypox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import os
import zipfile
from google.colab import files
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)
import matplotlib.pyplot as plt
import seaborn as sns

2.8.2


In [2]:
tf.random.set_seed(72)

In [4]:
drive.mount('/content/drive/')
zip_ref = zipfile.ZipFile("/content/drive/My Drive/monkeypox.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Mounted at /content/drive/


In [11]:
train=os.path.join("/tmp/Fold1/Fold1/Fold1/Train")
list_train=os.listdir(train)
test=os.path.join("/tmp/Fold1/Fold1/Fold1/Test")
list_test=os.listdir(test)
len(list_train),len(list_test)

(2, 2)

In [84]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    )
train_generator=train_datagen.flow_from_directory(directory="/tmp/Fold1/Fold1/Fold1/Train",target_size=(150,150)
                                                  ,class_mode="binary"
                                                  ,batch_size=64
                                                  
)


Found 2142 images belonging to 2 classes.


In [80]:
test_datagen=ImageDataGenerator()
test_generator=test_datagen.flow_from_directory(directory="/tmp/Fold1/Fold1/Fold1/Val",
                                                target_size=(150,150),
                                                class_mode="binary",
                                                
)

Found 420 images belonging to 2 classes.


In [78]:
val_datagen=ImageDataGenerator()
val_generator=val_datagen.flow_from_directory(directory="/tmp/Fold1/Fold1/Fold1/Train",
                                              target_size=(150,150),
                                              class_mode="binary")

Found 2142 images belonging to 2 classes.


In [40]:

model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [41]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 34, 34, 64)        36928     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 17, 17, 64)      

In [42]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
class Mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get("accuracy")>0.85):
      self.model.stop_training=True
callbacks=Mycallback()

In [86]:
history=model.fit(train_ganerator,epochs=40,callbacks=[callbacks])

Epoch 1/40
34/34 [==============================] - 89s 3s/step - loss: 0.2770 - accuracy: 0.8824


In [91]:
model.save("Monkeypox.h5")

KeyError: ignored

In [88]:
model.evaluate(test_generator)

14/14 [==============================] - 5s 320ms/step - loss: 187.7859 - accuracy: 0.6262


[187.78590393066406, 0.6261904835700989]

In [89]:
model.evaluate(train_generator)

34/34 [==============================] - 37s 1s/step - loss: 0.4773 - accuracy: 0.8464


[0.47726744413375854, 0.8464052081108093]

In [90]:
model.evaluate(val_generator)

67/67 [==============================] - 23s 342ms/step - loss: 84.9242 - accuracy: 0.7614


[84.92423248291016, 0.7614378929138184]